# An iterative method based on Sherman-Morrison-Woodbury regular splitting for nearly circulant matrices


### 18 July 2023
### Dimitrios Mitsotakis

This notebook serves as an APPENDIX to the paper "On iterative methods based on Sherman-Morrison-Woodbury regular splitting". For more information please refer to https://www.dmitsotakis.com/ or https://arxiv.org/abs/2305.10968

In [1]:
import numpy as np
import scipy as sp
import scipy.sparse as sps
import scipy.sparse.linalg as spsl
import numpy.linalg as npl
import scipy.linalg as spl
from numpy.fft import fft, ifft

## The SMW iteration

In [2]:
def SMW_iteration(M, N, b, x, tol = 1.e-5, maxit = 100):
    # M, N : matrices such as A = M - N
    # M : is the first column of the circulant matrix M
    # N : is the sparse matrix N
    # x : guess of the solution
    
    err = 1.0
    iters = 0
#     cc = fft(M)
    
    while (err > tol and iters < maxit):
        iters += 1
        c = N@x+b
#        xnew = fft(ifft(c)/cc).real
        xnew = spl.solve_circulant(M,c)
        err = npl.norm(xnew-x)
        x = np.copy(xnew)
        
    print('iterations required for convergence:', iters)
    
    return x

## The extrapolated SMW iteration

In [3]:
def eSMW_iteration(M, N, b, omega, x, tol = 1.e-5, maxit = 100):
    # M, N : matrices such as A = M - N
    # M : is the first column of the circulant matrix M
    # N : is the sparse matrix N
    # x : guess of the solution
    # omega : relaxation parameter usually omega=2/(l1+l2)
    # where l1 and l2 are the minimum and maximum eigenvalues respectively
    
    err = 1.0
    iters = 0
#    cc = fft(M)

    
    Bs = sps.csr_matrix((1-omega)*spl.circulant(M)+omega*N)
    bs = omega*b
    
    
    while (err > tol and iters < maxit):
        iters += 1
        c = Bs@x+bs
#        xnew = fft(ifft(c)/cc).real
        xnew = spl.solve_circulant(M,c)
        err = npl.norm(xnew-x)
        x = np.copy(xnew)
    
    
    print('iterations required for convergence:', iters)

    return x

## The block Gauss-Seidel-SMW iteration for $2\times 2$ block matrices

Solves linear systems with matrix
$$
\begin{pmatrix}
A_1 & B_1 \\
B_2 & A_2
\end{pmatrix}
=
\begin{pmatrix}
M_1-N_1 & B_1 \\
B_2 & M_2-N_2
\end{pmatrix}
$$

In [4]:
def block_SMW_iteration(M1, M2, N1, N2, B1, B2, b1, b2, x, y, tol = 1.e-5, maxit = 100):
    
    
    err = 1.0
    iters = 0
#    m1 = fft(M1)
#    m2 = fft(M2)
    
    while (err > tol and iters < maxit):
        iters += 1
        c1 = N1@x-B1@y+b1
#         xnew = fft(ifft(c1)/m1).real
        xnew = spl.solve_circulant(M1,c1)
        c2 = N2@y-B2@xnew+b2
#         ynew = fft(ifft(c2)/m2).real
        ynew = spl.solve_circulant(M2,c2)

        err = npl.norm(xnew-x)+npl.norm(ynew-y)
        x = np.copy(xnew)
        y = np.copy(ynew)
        
    print('iterations required for convergence:', iters)
    
    return x, y


## The block Gauss-Seidel-SMW iteration for a two-dimensional Poisson discretization 

Here the matrix $A$ is the $m^2\times m^2$ block matrix
$$
A=\begin{pmatrix}
D & I &   &   &\\
I & D & I &   & \\
  & \ddots & \ddots & \ddots & \\
  &   & I & D & I\\
  &   &   & I & D
\end{pmatrix}
$$
where $D=M-N=tridiag(1,-4,1)$ and $I$ is the $m \times m$ identity matrix.

In our implementation we provide $M$ and $N$

In [5]:
def mblocks_SMW_iteration(M, N, b, x, m, tol = 1.e-5, maxit = 100):
    # M, N : matrices such as D = M - N
    # M : is the first column of the circulant matrix M
    # N : is the sparse matrix N
    # x : guess of the solution
    # m : number of blocks in b
    
    err = 1.0
    iters = 0
    cc = fft(M)
    m2 = m**2
    
    while (err > tol and iters < maxit):
        iters += 1
        xold = np.copy(x)
        xi = x[0:m]
        xip1 = x[m:2*m]
        bi = b[0:m]
        c = N@xi-xip1+bi
#         x[0:m] = fft(ifft(c)/cc).real
        x[0:m] = spl.solve_circulant(M,c)
        for i in range(m,m2-m,m):
            xim1 = x[i-m:i]
            xi = x[i:i+m]
            xip1 = x[i+m:i+2*m]
            bi = b[i:i+m]
            c = N@xi-xip1-xip1+bi
#             x[i:i+m] = fft(ifft(c)/cc).real
            x[i:i+m] = spl.solve_circulant(M,c)
        xi = x[m2-m:m2]
        xim1 = x[m2-2*m:m2-m]
        bi = b[m2-m:m2]
        c = N@xi-xim1+bi
#         x[m2-m:m2] = fft(ifft(c)/cc).real
        x[m2-m:m2] = spl.solve_circulant(M,c)

        err = npl.norm(x-xold)
        
    print('iterations required for convergence:', iters)
    
    return x


## Example 1: Simple use of SMW iteration

In [25]:
# Setup of the problem - Assembly of matrices and vectors

A =  np.array([[5.,3.,2.,2.],
               [1.,4.,3.,2.],
               [2.,1.,4.,3.],
               [4.,2.,1.,5.]])

xexact = np.ones(4)
# xexact = np.random.rand(4)

b = A@xexact

N = sps.lil_matrix((4, 4))
N[0,0]=-1.0; N[3,0]=-1.0; N[0,3]=-1.0; N[3,3]=-1.0

M = (A + N.toarray())[:,0]

# x0 = initial guess

x0 = np.zeros(4)
x = SMW_iteration(M, N, b, x0, tol = 1.e-6, maxit = 100)

print('The error is:', npl.norm(x-xexact)) 

iterations required for convergence: 16
The error is: 1.0646762026574173e-07


## Example 2: Simple use of extrapolated SMW iteration

In [7]:
# Setup of the problem - Assembly of matrices and vectors

A =  np.array([[5.,3.,2.,2.],
               [1.,4.,3.,2.],
               [2.,1.,4.,3.],
               [4.,2.,1.,5.]])

xexact = np.ones(4)

b = A@xexact

N = sps.lil_matrix((4, 4))
N[0,0]=-1.0; N[3,0]=-1.0; N[0,3]=-1.0; N[3,3]=-1.0

M = (A + N.toarray())[:,0]

omega = 0.7

# x0 = initial guess

x0 = np.zeros(4)
x = eSMW_iteration(M, N, b, omega, x0, tol = 1.e-6, maxit = 100)

print('The error is:', npl.norm(x-xexact)) 

iterations required for convergence: 13
The error is: 3.944926335045367e-07


## Example 3: A more demanding example for the SMW iteration

In [8]:
# Setup of the problem - Assembly of matrices and vectors

n = 10000 # This is the dimension of the system
M = np.zeros(n)
M[0] = 16
M[1] = -5
M[-1] = -5
M = M/6.0

N = sps.lil_matrix((n,n))
N[0,0]=8.0; N[n-1,n-1]=8.0; N[0,-1]=-5.0; N[n-1,0]=-5.0;
N = N/6.0

A = spl.circulant(M) - N.toarray()
xexact = np.ones(n)
b = A@xexact

# x0 = initial guess

x0 = np.zeros(n)
x = SMW_iteration(M, N, b, x0, tol = 1.e-8, maxit = 100)

print('The error is:', npl.norm(x-xexact)) 

iterations required for convergence: 18
The error is: 2.403946595612752e-09


## Example 4: A more demanding example for the eSMW iteration

In [9]:
# Setup of the problem - Assembly of matrices and vectors

n = 4 # This is the dimension of the system
M = np.zeros(n)
M[0] = 16
M[1] = -5
M[-1] = -5
M = M/6.0

N = sps.lil_matrix((n,n))
N[0,0]=8.0; N[n-1,n-1]=8.0; N[0,-1]=-5.0; N[n-1,0]=-5.0;
N = N/6.0

A = spl.circulant(M) - N.toarray()
xexact = np.ones(n)
b = A@xexact

omega = 1.2

# x0 = initial guess

x0 = np.zeros(n)
x = eSMW_iteration(M, N, b, omega, x0, tol = 1.e-8, maxit = 100)

print('The error is:', npl.norm(x-xexact)) 

iterations required for convergence: 14
The error is: 6.571838406330155e-10


## Example 5: A case where other methods will more likely fail

In this example we consider a dense nearly circulant (not symmetric) matrix with 1,000,000 entries. The SMW iteration converges extremely fast and we only store the first column of the matrix along with the sparse matrix $N$.

In [10]:
from numpy.random import rand

n = 1000000

M = 1.0+rand(n)
xexact = np.ones((n,1))
N = sps.lil_matrix((n, n))
N[0,0]=-1.0; N[-1,0]=-1.0; N[0,-1]=-1.0; N[-1,-1]=-1.0;

s = np.sum(M)
b = np.ones((n,1))*s
b[0] -= N[0,0] + N[0,-1]
b[-1] -= N[-1,0] + N[-1,-1]

xexact = np.ones((n,1))
x0 = np.zeros((n,1))

# A = spl.circulant(M)-N
# print('condition number=',npl.cond(A))
# b = A@xexact


x = SMW_iteration(M, N, b, x0, tol = 1.e-8, maxit = 100)

print('The error is:', npl.norm(x-xexact)) 

iterations required for convergence: 4
The error is: 3.041231118794826e-09


## Example 6: Application to block matrices

In [11]:
n=100
M1 = np.zeros(n)
M1[0]=16
M1[1]=-5
M1[-1]=-5
M = spl.circulant(M1)

N = np.zeros((n,n))
N[0,0]=8; N[n-1,n-1]=8; N[0,n-1]=-5; N[n-1,0]=-5;

A = M - N

xexact = np.ones(n)
x = np.zeros(n)
b = A@xexact

B1 = np.zeros(n)
B1[0]=0
B1[1]=-1./2.
B1[-1]=1./2.
B = spl.circulant(B1)
B[0,-1]=0
B[-1,0]=0

S = np.block([[A,B],
              [B,A]])

xexact = np.ones(2*n)
x = np.zeros(n)
y = np.zeros(n)
b = S@xexact
b1=b[:n]
b2=b[n:]

N1 = sps.csr_matrix(N) 
B1 = sps.csr_matrix(B)
    
x, y = block_SMW_iteration(M1, M1, N1, N1, B1, B1, b1, b2, x, y, tol = 1.e-8, maxit = 100)

print('The error is:', npl.norm(x-xexact[:n])+npl.norm(y-xexact[n:]) )

iterations required for convergence: 38
The error is: 1.9193835368774438e-08


## Example 7: Application to two-dimensional Poisson equation

In [12]:
m = 20
m2 = m**2

M = np.zeros(m)

M[0] = -4.0; M[1] = 1.0; M[-1] = 1.0;

N = sps.lil_matrix((m,m))
N[0,-1] = 1.0
N[-1,0] = 1.0

# assembly of rhs
b = np.zeros(m2)
b1 = -np.ones(m)
b1[0] = -2.0
b1[-1] = -2.0
bi = np.zeros(m)
bi[0] = -1.0
bi[-1] = -1.0
for i in range(1,m-1):
    b[i*m:i*m+m]=bi[:]
b[0] = -2.0
b[1] = -1.0
b[0:m]=b1[:]
b[m2-m:m2]=b1[:]

xexact = np.ones(m**2)

x = np.zeros(m2)

x = mblocks_SMW_iteration(M, N, b, x, m, tol = 1.e-8, maxit = 10000)

print('The error is:', npl.norm(x-xexact)) 

iterations required for convergence: 70
The error is: 2.5126624349556545e-09
